# India Cotton Analysis - EDA Notebook

In [2]:
import pandas as pd
import numpy as np
import datetime
import pickle
import unicodedata

import sqlalchemy
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
import re
from ipywidgets import interact, IntSlider, fixed

import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
from shapely import wkt
from shapely.prepared import prep

from scipy.optimize import minimize
from geopy.distance import geodesic

warnings.filterwarnings('ignore')
np.random.seed(29)

### Load in Data
- Geographic Data : [link](https://data.humdata.org/dataset/geoboundaries-admin-boundaries-for-india?force_layout=desktop)
- Climate Data : [link](https://datacatalog.worldbank.org/search/dataset/0061895/India-Climatic-Data)
- Cotton Prices : [link](https://fred.stlouisfed.org/series/PCOTTINDUSDM#0)

In [4]:
### Geographic Data
path = './data/geographic_data/geoBoundaries-IND-ADM1_simplified.geojson'
india_state_geos = gpd.read_file(path)
india_state_geos.index.name = 'asdf_id'

path = './data/geographic_data/geoBoundaries-IND-ADM2_simplified.geojson'
india_admin_geos = gpd.read_file(path)
india_admin_geos.index.name = 'asdf_id'

### Climate Data
path = './data/geographic_data/GeoQuery_India_ADM2_results.csv'
climate_data = pd.read_csv(path).set_index('asdf_id')

path = './data/agricultural_data/crop_yield.csv'
crop_yields = pd.read_csv(path)

### Cotton Prices
path = './data/economic_data/cotton_prices_1990_2024.csv'
cotton_prices = pd.read_csv(path)
cotton_prices['DATE'] = pd.to_datetime(cotton_prices['DATE'])

In [5]:
cd_years = []

for col in climate_data.columns:
    try:
        yr = int(col.split('.')[-2])
        if (yr not in cd_years) & (yr < 2024):
            cd_years.append(yr)
    ### For named columns (e.g. 'Shape_Area')
    except:
        pass

### Data Cleaning

##### `india_state_geos` \& `india_admin_geos`

In [8]:
def remove_accents(input_str):
    '''
    Description
    --------------------------------------------------
    Removes diacritical marks (accents) from a given
    string.
    
    Inputs
    --------------------------------------------------
    + input_str : str; the input string from which to
      remove accents
    
    Outputs
    --------------------------------------------------
    + Returns a string with accents removed.
    '''
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [9]:
### This ensures consistency between the state names in 
### crop_yields & india_state_geos
india_state_geos['shapeName'] = india_state_geos['shapeName'].str.lower().apply(remove_accents)

In [10]:
def determine_state(polygon, state_gdf):
    '''
    Description
    --------------------------------------------------
    Determines the state in which a given polygon is 
    located by checking intersections with state 
    geometries.
    
    Inputs
    --------------------------------------------------
    + polygon : shapely.geometry.Polygon; the polygon
      for which to determine the state
    + state_gdf : geopandas.GeoDataFrame; a 
      GeoDataFrame containing state geometries and 
      their corresponding names
    
    Outputs
    --------------------------------------------------
    + Returns the name of the state in which the 
      polygon is located, or np.nan if no 
      intersection is found.
    '''
    state = None
    for i in range(len(state_gdf)):
        state_geo = state_gdf.loc[i, 'geometry']
        if polygon.intersects(state_geo):
            state = state_gdf.loc[i, 'shapeName']
            return state
    if state == None:
        return np.nan

In [11]:
### This gets the state name for each of the 
### administrative areas
india_admin_geos['shapeName'] = india_admin_geos['shapeName'].str.lower().apply(remove_accents)
india_admin_geos['stateName'] = india_admin_geos['geometry'].apply(lambda polygon : determine_state(polygon, india_state_geos))

##### `crop_yields` \& `cotton_yields`
According to the USDA, Maharashtra, Gujarat, Telangana, Rajasthan, & Haryana produce about 80 percent of the cotton in India (which produces about 21 percent of the world's cotton). [source](https://ipad.fas.usda.gov/cropexplorer/cropview/commodityView.aspx?cropid=2631000)

In [13]:
cotton_states = ['maharashtra', 'gujarat', 'telangana', 'rajasthan', 'haryana']
crop_yields['District_Name'] = crop_yields['District_Name'].str.lower().apply(remove_accents)
crop_yields['State_Name'] = crop_yields['State_Name'].str.lower().apply(remove_accents)
condition = (crop_yields['Crop'] == 'Cotton(lint)') & (crop_yields['State_Name'].isin(cotton_states))
cotton_yields = crop_yields[condition].reset_index(drop = True)
cotton_yields['Season'] = cotton_yields['Season'].str.strip()

### Ensures that values for yr X for both dfs
min_yr = max(min(cotton_prices['DATE'].dt.year), min(cd_years))
max_yr = min(max(cotton_prices['DATE'].dt.year), max(cd_years))

cotton_districts = cotton_yields['District_Name'].unique()
cotton_district_ids = india_admin_geos[india_admin_geos['shapeName'].isin(cotton_districts)].index
cotton_district_ids

Index([ 87,  88,  91,  93,  94,  95,  96,  97,  98,  99, 100, 101, 103, 104,
       105, 107, 110, 111, 112, 113, 115, 118, 119, 165, 196, 197, 198, 199,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 447, 449, 450, 451, 452, 454, 455, 456, 457, 458, 459,
       460, 461, 462, 463, 464, 466, 467, 469, 470, 472, 473, 477, 482, 514,
       515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528,
       529, 530, 531, 532, 533, 535, 537, 538, 604, 646, 647, 648, 650, 651,
       652],
      dtype='int64', name='asdf_id')

##### `climate_data` $\rightarrow$ `cd_rel` (Relevant Climate Data)

In [15]:
### This  will allow climate_data to be joined with india_geos
### in the future if need be
climate_data['shapeID'] = [i.split("-")[-1] for i in climate_data['shapeID']]

In [237]:
d1 = climate_data.filter(like='co2').filter(like='mean').isna().sum().sort_values().reset_index()#.groupby('asdf_id').filter(lambda x: not x.isna().any().any())


In [239]:
d1[0].value_counts(dropna = False)

0
0    77
Name: count, dtype: int64

In [16]:
desired_cols = ['Shape_Area', 'Shape_Length', 'shapeID', 'shapeName']

In [17]:
categories = []

for col in climate_data.columns:
    try:
        cat = col.split('.')[-3]
        if cat not in categories:
            categories.append(cat)
    ### For named columns (e.g. 'Shape_Area')
    except:
        pass

In [18]:
# ### If yearly cols wanted uncomment this
# desired_categories = {'cru_ts_405_tmp_yearly_mean' : 'yearly_mean_temp', 'cru_ts_405_tmp_monthly_mean' : 'monthly_mean_temp',
#                       'cru_ts_405_pre_yearly_mean' : 'yearly_mean_precip', 'cru_ts_405_pre_monthly_mean' : 'monthly_mean_precip',
#                       'oco2_v10r_xco2_monthly' : 'co2_concentration', 'ltdr_avhrr_ndvi_v5_monthly' : 'max_ndvi'}

### Only contains monthly cols
desired_categories = {'cru_ts_405_tmp_monthly_mean' : 'monthly_mean_temp', 'cru_ts_405_pre_monthly_mean' : 'monthly_mean_precip',
                      'oco2_v10r_xco2_monthly' : 'co2_concentration', 'ltdr_avhrr_ndvi_v5_monthly' : 'max_ndvi'}

In [19]:
valid_cols = []

### Gets Valid Years
for col in climate_data.columns:
    try:
        yr = col.split('.')[-2]
        ### For years like 2000
        condition1 = (int(yr) > min_yr - 10) & (int(yr) <= max_yr)
        ### For years like 200010 ==> Oct 2000
        condition2 = (int(yr) > (min_yr - 10) * 100) & (int(yr) <= max_yr * 100)
        if condition1 or condition2:
            valid_cols.append(col)
    ### For named columns (e.g. 'Shape_Area')
    except:
        pass


v2 = []
### Gets Valid Categories
for col in valid_cols:
    cat = col.split('.')[-3]
    mode = col.split('.')[-1]
    if (cat in desired_categories.keys()) and (mode == 'mean'):
        v2.append(col)

valid_cols = v2
valid_cols.sort()
desired_cols.extend(valid_cols)

### For memory purposes
v2 = None

### Prevents valid_cols from being added
### multiple times to desired_cols
if len(desired_cols) < len(valid_cols):
    desired_cols.extend(valid_cols)

In [20]:
### cd_rel ==> Relevant Climate Data
cd_rel = climate_data[climate_data.index.isin(cotton_district_ids)][valid_cols]
cd_rel = cd_rel.melt(var_name = 'cat', ignore_index = False).reset_index()
cd_rel['year'] = cd_rel['cat'].str.split('.').str[-2].str[:-2].astype(int)
cd_rel['month'] = cd_rel['cat'].str.split('.').str[-2].str[-2:].astype(int)
cd_rel['cat'] = cd_rel['cat'].apply(lambda cat : desired_categories[cat.split('.')[0]])

### Feature Engineering

##### `cd_rel`

In [23]:
def get_xytd_changes(df, years_back):
    '''
    Description
    --------------------------------------------------
    Calculates the percentage change in values over a
    specified number of years back for each row in 
    the DataFrame.
    
    Inputs
    --------------------------------------------------
    + df : pandas.DataFrame; the input DataFrame 
      containing columns 'asdf_id', 'cat', 'year', 
      'month', and 'value'
    + years_back : int; the number of years back to 
      compare the current value against
    
    Outputs
    --------------------------------------------------
    + Returns a list of percentage changes, or NaN if
      the comparison value does not exist.
    '''
    ### Time Complexity : O(n^2)
    changes = []
    for i in range(len(df)):
        row = df.loc[i, :]
        asdf_id = row['asdf_id']
        cat = row['cat']
        curr_yr = row['year']
        curr_month = row['month']
        curr_val = row['value']
        
        ### If old_val exists
        try:
            condition = (df['asdf_id'] == asdf_id) & (df['cat'] == cat) & (df['year'] == curr_yr - years_back) & (df['month'] == curr_month)
            old_val = df.loc[condition, 'value']
            change = (100 * (curr_val - old_val) / old_val).values[0]
            changes.append(change)
        except:
            changes.append(np.nan)
    
    return changes

In [24]:
### Commented because it takes a long time to run 
ytd_change1 = get_xytd_changes(cd_rel, years_back = 1)
ytd_change3 = get_xytd_changes(cd_rel, years_back = 3)
ytd_change5 = get_xytd_changes(cd_rel, years_back = 5)
ytd_change10 = get_xytd_changes(cd_rel, years_back = 10)
cd_rel['1yr_ytd_change'] = ytd_change1
cd_rel['3yr_ytd_change'] = ytd_change3
cd_rel['5yr_ytd_change'] = ytd_change5
cd_rel['10yr_ytd_change'] = ytd_change10

In [267]:
# ### This is commented out as it is provided in 
# ### ./data/agricultural_data
# path = './data/agricultural_data/cd_rel_raw.csv'
# cd_rel.to_csv(path, index = False)

In [744]:
### Uncomment to load in the data
path = './data/agricultural_data/cd_rel_raw.csv'
cd_rel = pd.read_csv(path)

In [746]:
### Too many NaNs values make 'mac_ndvi' and 'co2_concentration' unusable for the current project
cd_rel = cd_rel[~(cd_rel['cat'].isin(['max_ndvi', 'co2_concentration']))].reset_index(drop = True)

### Only Need entries from Jan 1990 through Dec 2019
cd_rel = cd_rel[cd_rel['year'] >= 1990].reset_index(drop = True)

In [748]:
### Districts with No NaNs for the categories of interest up to 10 yr change
cd10yr = cd_rel.groupby('asdf_id').filter(lambda x: not x.isna().any().any())['asdf_id'].unique()

### Districts with No NaNs for the categories of interest up to 5 yr change
cd5yr = cd_rel.drop(columns = ['10yr_ytd_change']).groupby('asdf_id').filter(lambda x: not x.isna().any().any())['asdf_id'].unique()

### Districts with No NaNs for the categories of interest up to 3 yr change
cd3yr = cd_rel.drop(columns = ['10yr_ytd_change', 
                               '5yr_ytd_change']).groupby('asdf_id').filter(lambda x: not x.isna().any().any())['asdf_id'].unique()

### Districts with No NaNs for the categories of interest up to 1 yr change
cd1yr = cd_rel.drop(columns = ['10yr_ytd_change',
                               '5yr_ytd_change',
                               '3yr_ytd_change']).groupby('asdf_id').filter(lambda x: not x.isna().any().any())['asdf_id'].unique()
print(f'Number of Clean Districts:\nUp to Ten Years: {len(cd10yr)}\nUp to Five Years: {len(cd5yr)}\nUp to Three Years: {len(cd3yr)}\nUp to One Year: {len(cd1yr)}')

Number of Clean Districts:
Up to Ten Years: 86
Up to Five Years: 86
Up to Three Years: 88
Up to One Year: 91


*Note*: (a) Since there are the same amount of "clean" districts at the 5ytd and 10ytd change levels, I use the same df for both. (b) I relplace all the `np.inf` values with **monthly medians**.

In [ ]:
def replace_inf(row, val_col, monthly_meds):
    '''
    Description
    --------------------------------------------------
    Replaces infinity values in a specified column 
    of a DataFrame row with corresponding median 
    values from another DataFrame.
    
    Inputs
    --------------------------------------------------
    + row : pandas.Series; a row from the DataFrame 
      being processed
    + val_col : str; the name of the column in which
      to check for infinity values
    + monthly_meds : pandas.DataFrame; a DataFrame 
      containing median values indexed by 
      (year, month, category)
    
    Outputs
    --------------------------------------------------
    + Returns the original value if it is not 
      infinity, otherwise returns the median value 
      for the corresponding year, month, and category.
    '''
    val = row[val_col]
    if (val == np.inf) or (val == -np.inf):
        c = row['cat']
        y = row['year']
        m = row['month']
        new_val = monthly_meds.loc[(y, m, c), val_col]
        return new_val
    else:
        return val

In [952]:
cd_rel_temp = pd.DataFrame(columns = ['year', 'month', 'cat', 'value'])
suffixes = {'value' : 'val', '1yr_ytd_change' : '1ytd', '3yr_ytd_change' : '3ytd',
            '5yr_ytd_change' : '5ytd', '10yr_ytd_change' : '10ytd'}

### All ytd changes
### Makes sure it only runs once
if not 'cd_rel_full' in locals():
    ### Needed to impute np.inf values
    monthly_meds = cd_rel.drop('asdf_id', axis = 1).groupby(['year', 'month', 'cat']).median()

    for i in range(len(suffixes)):
        s = list(suffixes.keys())[i]
        sub_df = cd_rel[cd_rel['asdf_id'].isin(cd5yr)][['asdf_id', 'cat', 'year', 'month', s]]
        
        ### Replaces np.inf values with monthly medians
        sub_df[s] = sub_df.apply(lambda row : replace_inf(row, s, monthly_meds), axis = 1)
        
        sub_df['cat'] = sub_df.apply(lambda row : f"{row['cat']}_{row['asdf_id']}_{suffixes[s]}", axis = 1)
        sub_df = sub_df.rename(columns = {s : 'value'})

        ### asdf_id col becomes redundant after cat reformatted
        sub_df = sub_df.drop(columns = ['asdf_id'])

        cd_rel_temp = pd.concat([cd_rel_temp, sub_df])

    cd_rel_full = cd_rel_temp.pivot(columns = 'cat', index = ['year', 'month'], values = 'value')

    cd_rel_full.columns.name = None

    cd_rel_full = cd_rel_full[sorted(cd_rel_full.columns)].reset_index()
    
    path = './data/agricultural_data/cd_rel_clean_full.csv'
    cd_rel_full.to_csv(path, index = False)

    ### For memory purposes
    cd_rel_temp = None

    ### Up to 3ytd
    ### Makes sure it only runs once
    for i in range(len(suffixes) - 2):
        s = list(suffixes.keys())[i]
        sub_df = cd_rel[cd_rel['asdf_id'].isin(cd3yr)][['asdf_id', 'cat', 'year', 'month', s]]
        
        ### Replaces np.inf values with monthly medians
        sub_df[s] = sub_df.apply(lambda row : replace_inf(row, s, monthly_meds), axis = 1)
        
        sub_df['cat'] = sub_df.apply(lambda row : f"{row['cat']}_{row['asdf_id']}_{suffixes[s]}", axis = 1)
        sub_df = sub_df.rename(columns = {s : 'value'})

        ### asdf_id col becomes redundant after cat reformatted
        sub_df = sub_df.drop(columns = ['asdf_id'])

        cd_rel_temp = pd.concat([cd_rel_temp, sub_df])

    cd_rel_3ytd = cd_rel_temp.pivot(columns = 'cat', index = ['year', 'month'], values = 'value')

    cd_rel_3ytd.columns.name = None

    cd_rel_3ytd = cd_rel_3ytd[sorted(cd_rel_3ytd.columns)].reset_index()
    
    path = './data/agricultural_data/cd_rel_clean_3ytd.csv'
    cd_rel_3ytd.to_csv(path, index = False)

    ### For memory purposes
    cd_rel_temp = None

    ### Up to 1ytd
    ### Makes sure it only runs once
    for i in range(len(suffixes) - 3):
        s = list(suffixes.keys())[i]
        sub_df = cd_rel[cd_rel['asdf_id'].isin(cd1yr)][['asdf_id', 'cat', 'year', 'month', s]]
        
        ### Replaces np.inf values with monthly medians
        sub_df[s] = sub_df.apply(lambda row : replace_inf(row, s, monthly_meds), axis = 1)
        
        sub_df['cat'] = sub_df.apply(lambda row : f"{row['cat']}_{row['asdf_id']}_{suffixes[s]}", axis = 1)
        sub_df = sub_df.rename(columns = {s : 'value'})

        ### asdf_id col becomes redundant after cat reformatted
        sub_df = sub_df.drop(columns = ['asdf_id'])

        cd_rel_temp = pd.concat([cd_rel_temp, sub_df])

    cd_rel_1ytd = cd_rel_temp.pivot(columns = 'cat', index = ['year', 'month'], values = 'value')

    cd_rel_1ytd.columns.name = None

    cd_rel_1ytd = cd_rel_1ytd[sorted(cd_rel_1ytd.columns)].reset_index()
    
    path = './data/agricultural_data/cd_rel_clean_1ytd.csv'
    cd_rel_1ytd.to_csv(path, index = False)

    ### For memory purposes
    cd_rel_temp = None